In [119]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from os.path import expanduser
import re
from nltk.stem.porter import PorterStemmer
import nltk

In [120]:
stop_words = [word.strip() for word in open('stop_words.txt').readlines()]

In [121]:
def stemming_tokenizer(str_input):
    porter_stemmer = PorterStemmer()
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [122]:
with open('dems.txt', 'r',encoding="utf-8") as file:
    dem_text = [line.strip('\n') for line in file]
with open('gop.txt', 'r',encoding="utf-8") as file:
    gop_text = [line.strip('\n') for line in file]
with open('NonPolitical.txt', 'r',encoding="utf-8") as file:
    nonp_text = [line.strip('\n') for line in file]

In [123]:
dem=np.array(dem_text)
gop=np.array(gop_text)
nonp=np.array(nonp_text)

In [124]:
dem_df = pd.DataFrame({'tweet': dem})
dem_df['label']=0
gop_df = pd.DataFrame({'tweet': gop})
gop_df['label']=1
nonp_df = pd.DataFrame({'tweet': nonp})
nonp_df['label']=2

In [414]:
tweets=[dem_df,gop_df,nonp_df]
tweets_df=pd.concat(tweets,ignore_index=True)

In [428]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc=re.sub(r'-',' ',doc).strip()
    doc=re.sub(r'#','',doc).strip() #removing #symbol
    #doc=re.sub(r'#\S+','',doc).strip() #removing #symbol
    doc=re.sub(r'RT[\s]+','',doc).strip()
    doc = re.sub(r'http[a-zA-Z]*\S+', '', doc).strip()
    doc=re.sub(r'@[A-Za-z0-9]+','',doc).strip() #remove mentions
    doc=re.sub(r'[?|$|.|!|;|:|&|"|,|""|*|-|(|)_]','',doc).strip()
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [429]:
doc="Are you registered to vote in the August 7th primary? Check to make sure you're registered and your address is up to date! Click here: https://t.co/gvmxl7JOqd. #RegisterToVote #ElectionsMatter https://t.co/yZVOSYEsCj"
doc=re.sub(r'-',' ',doc).strip()
doc=re.sub(r'#\S+','',doc).strip() #removing #symbol
doc=re.sub(r'RT[\s]+','',doc).strip()
doc = re.sub(r'http[a-zA-Z]*\S+', '', doc).strip()
doc=re.sub(r'@[A-Za-z0-9]+','',doc).strip() #remove mentions
doc=re.sub(r'RT[\s]+','',doc).strip()
doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
doc=re.sub(r'[?|$|.|!|;|:|&|"|,|""|*|-|(|)_]','',doc).strip()
doc = doc.lower()
doc = doc.strip()
doc
    

'are you registered to vote in the august th primary check to make sure youre registered and your address is up to date click here'

In [128]:
norm_dem=normalize_corpus(dem_df['tweet'])
norm_gop=normalize_corpus(gop_df['tweet'])
norm_nonp=normalize_corpus(nonp_df['tweet'])

In [129]:
norm_tweets=np.concatenate((norm_dem, norm_gop,norm_nonp), axis=None)

In [130]:
from gensim.models.fasttext import FastText

wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in norm_tweets]

# Set values for various parameters
feature_size = 100    # Word vector dimensionality  
window_context = 30          # Context window size                                                                                    
min_word_count = 5   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words


ft_model = FastText(tokenized_corpus, size=feature_size, window=window_context, 
                    min_count=min_word_count,sample=sample, sg=1, iter=50)

In [131]:
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
ft_feature_array = averaged_word_vectorizer(corpus=tokenized_corpus, model=ft_model,
                                             num_features=feature_size)
tweet_ft=pd.DataFrame(ft_feature_array)
tweet_ft

C:\Users\pavan\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.074526,0.080293,0.187467,0.090009,-0.044608,-0.387754,-0.002318,-0.199358,0.099426,-0.017642,...,-0.145355,0.029845,0.057869,0.246256,-0.003921,-0.165760,0.213546,-0.086376,-0.176870,-0.200971
1,-0.239091,0.233052,0.363133,0.273586,-0.054622,-0.151443,-0.133992,0.105343,-0.069890,-0.059064,...,-0.118085,0.171875,-0.281486,0.088595,0.273388,0.085581,0.183766,0.228951,-0.015789,0.026417
2,-0.074666,-0.015435,0.244748,0.011251,0.026392,-0.099905,-0.120880,-0.086266,-0.039675,-0.002827,...,-0.161749,-0.086074,0.161439,0.136028,0.103543,-0.220837,0.197441,-0.199242,-0.293802,0.047402
3,-0.182422,0.181273,0.133351,0.105507,0.032164,-0.178636,0.160950,-0.222687,-0.515046,-0.095063,...,-0.211235,0.109812,-0.310593,0.045893,0.359497,0.058517,0.166160,-0.248691,-0.142845,0.090452
4,-0.077430,0.071810,0.073115,-0.073652,-0.221686,-0.402776,0.022441,-0.350587,-0.125873,-0.022526,...,-0.242557,0.276498,-0.059448,0.141366,0.031232,0.016932,0.104832,-0.036967,-0.012743,-0.196125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51259,-0.006271,-0.073379,0.093922,0.021232,-0.011982,-0.110760,-0.037173,0.171148,-0.289264,0.159753,...,0.136248,0.060517,0.041330,-0.057060,0.085408,-0.249345,0.212613,-0.063568,-0.009904,-0.091655
51260,0.104767,-0.013414,0.431981,0.119724,-0.124743,-0.682199,0.366728,0.117721,-0.181722,-0.028172,...,0.007680,0.165578,0.175792,0.498462,0.296007,-0.247617,0.015344,-0.260184,0.250670,0.013284
51261,-0.065076,-0.058622,0.224754,0.098286,-0.259205,-0.469793,-0.008166,-0.206598,-0.010651,-0.061007,...,-0.345821,0.065550,-0.243491,0.065021,-0.001672,0.135027,0.403328,-0.188708,-0.033943,-0.019447
51262,0.172080,0.014199,0.304128,0.147693,-0.115717,-0.287600,0.181923,0.250200,-0.366083,0.066388,...,0.141367,0.235300,0.207527,0.289264,0.228060,-0.234011,0.220885,-0.165652,0.188325,-0.190407


In [132]:
# tweet_ft.to_csv("FastText.csv",index=False)

In [133]:
tweetft_df=pd.concat([tweet_ft,tweets_df], axis=1)

In [134]:
tweetft_df

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,tweet,label
0,-0.074526,0.080293,0.187467,0.090009,-0.044608,-0.387754,-0.002318,-0.199358,0.099426,-0.017642,...,0.057869,0.246256,-0.003921,-0.165760,0.213546,-0.086376,-0.176870,-0.200971,This week @senatemajldr said workers don’t nee...,0
1,-0.239091,0.233052,0.363133,0.273586,-0.054622,-0.151443,-0.133992,0.105343,-0.069890,-0.059064,...,-0.281486,0.088595,0.273388,0.085581,0.183766,0.228951,-0.015789,0.026417,Health care professionals are on the front lin...,0
2,-0.074666,-0.015435,0.244748,0.011251,0.026392,-0.099905,-0.120880,-0.086266,-0.039675,-0.002827,...,0.161439,0.136028,0.103543,-0.220837,0.197441,-0.199242,-0.293802,0.047402,RT @SeemaNanda: Good to see @Google signal a c...,0
3,-0.182422,0.181273,0.133351,0.105507,0.032164,-0.178636,0.160950,-0.222687,-0.515046,-0.095063,...,-0.310593,0.045893,0.359497,0.058517,0.166160,-0.248691,-0.142845,0.090452,Republicans keep admitting that voter suppress...,0
4,-0.077430,0.071810,0.073115,-0.073652,-0.221686,-0.402776,0.022441,-0.350587,-0.125873,-0.022526,...,-0.059448,0.141366,0.031232,0.016932,0.104832,-0.036967,-0.012743,-0.196125,RT @SpeakerPelosi: The Congress has so far pas...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51259,-0.006271,-0.073379,0.093922,0.021232,-0.011982,-0.110760,-0.037173,0.171148,-0.289264,0.159753,...,0.041330,-0.057060,0.085408,-0.249345,0.212613,-0.063568,-0.009904,-0.091655,RT @RecordingAcad: Who is nominated in the Gen...,2
51260,0.104767,-0.013414,0.431981,0.119724,-0.124743,-0.682199,0.366728,0.117721,-0.181722,-0.028172,...,0.175792,0.498462,0.296007,-0.247617,0.015344,-0.260184,0.250670,0.013284,RT @WSJ: Instagram users can now turn off comm...,2
51261,-0.065076,-0.058622,0.224754,0.098286,-0.259205,-0.469793,-0.008166,-0.206598,-0.010651,-0.061007,...,-0.243491,0.065021,-0.001672,0.135027,0.403328,-0.188708,-0.033943,-0.019447,.@valiswiser is on a mission to help people ov...,2
51262,0.172080,0.014199,0.304128,0.147693,-0.115717,-0.287600,0.181923,0.250200,-0.366083,0.066388,...,0.207527,0.289264,0.228060,-0.234011,0.220885,-0.165652,0.188325,-0.190407,RT @TechCrunch: Instagram fights abuse with co...,2


In [135]:
from sklearn.model_selection import train_test_split
x=tweetft_df.drop(['tweet','label'],axis=1)
y=tweetft_df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [136]:
from sklearn.naive_bayes import BernoulliNB
naive_bayes = BernoulliNB()
model = naive_bayes.fit(x_train, y_train)
y_predictions = model.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predictions)

0.6546504369538078

In [137]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state = 42) 
log_classifier = LogisticRegression(multi_class='multinomial',solver ='newton-cg')
log_classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [138]:
y_pred = log_classifier.predict(X_test)

In [139]:
log_classifier.score(X_test, y_test) 

0.7510143570536829

In [140]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3572, 1026,  241],
       [ 993, 3330,  377],
       [ 213,  341, 2723]], dtype=int64)

In [141]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[3574,  997,  268],
       [1021, 3268,  411],
       [ 216,  315, 2746]], dtype=int64)

In [142]:
model.score(X_test, y_test) 

0.74812734082397

In [143]:
similar_words = {search_term: [item[0] for item in ft_model.wv.most_similar([search_term], topn=5)]
                  for search_term in ['gop', 'dem', 'vote', 'attack', 'administration', 'voters','win','trump']}
similar_words

{'gop': ['republicans', 'republican', 'senators', 'vulnerable', 'senate'],
 'dem': ['dems', 'democratic', 'democrat', 'pelosi', 'solicited'],
 'vote': ['voting', 'registered', 'polls', 'ballot', 'confirm'],
 'attack': ['attacks', 'attacked', 'horrific', 'attacking', 'cowardly'],
 'administration': ['administrations', 'trump', 'admin', 'president', 'cruel'],
 'voters': ['electi', 'republicans', 'polls', 'ballot', 'november'],
 'win': ['chance', 'compete', 'victory', 'tickets', 'winner'],
 'trump': ['president', 'donald', 'trumps', 'administration', 'presidency']}

In [144]:
import fasttext

In [588]:
tweets=[dem_df,gop_df,nonp_df]
tweets_df=pd.concat(tweets,ignore_index=True)

In [589]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc=re.sub(r'-',' ',doc).strip()
    #doc=re.sub(r'#\S+','',doc).strip() #removing #symbol
    doc=re.sub(r'RT[\s]+','',doc).strip()
    doc = re.sub(r'http[a-zA-Z]*\S+', '', doc).strip()
    #doc=re.sub(r'@[A-Za-z0-9]+','',doc).strip() #remove mentions
    doc=re.sub(r'[?|$|.|!|;|:|&|"|,|""|*|-|(|)|#|_|@]','',doc).strip()
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [590]:
tweets_df["tweet"]=tweets_df["tweet"].apply(normalize_document)

In [591]:
def add_prefix(colum):
    return "__label__"+str(colum)

In [592]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p)) #Precision
    print("R@{}\t{:.3f}".format(1, r)) #Recall

In [593]:
x=tweets_df["tweet"]
y=tweets_df["label"]

In [594]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [595]:
train_df=pd.DataFrame()
train_df["value"]=x_train
train_df["label"]=y_train
train_df

,value,label
37756,great article taxreformer create jobs act make...,1
2760,tune watch tomperez talk trevornoah 2020 upcom...,0
50153,finding adventure daily commute 👀 whpadventure,2
27055,honor president realdonaldtrump reaffirm every...,1
34271,mccormackjohn romney brings libya ask ' politi...,1
...,...,...
11284,17 year incumbent cathymcmorris coward sought ...,0
44732,playavengers marvel ultimate alliance 3 black ...,2
38158,cbo projects roughly 2 % real gdp growth next ...,1
860,seventh demdebate next week ’ still time host ...,0


In [596]:
test_df=pd.DataFrame()
test_df["value"]=x_test
test_df["label"]=y_test
test_df

,value,label
38597,come dray rim best come prepared 💪,2
22907,realdonaldtrump administration unleashing full...,1
13695,repcolinallred first wave korean immigrants ar...,0
29642,brave men women uniform away families christma...,1
36671,prevention injury disease plays vital role hea...,1
...,...,...
1533,michigandems ' right folks today must register...,0
48085,endangered animal portraits created one hole t...,2
41469,chasecenter case missed ’ 👀 back first warrior...,2
9002,support continues tank disapproval rating cont...,0


In [597]:
train_df["value"]=train_df["label"].apply(add_prefix)+" "+train_df["value"]
test_df["value"]=test_df["label"].apply(add_prefix)+" "+test_df["value"]

In [598]:
train_df["value"]

37756    __label__1 great article taxreformer create jo...
2760     __label__0 tune watch tomperez talk trevornoah...
50153    __label__2 finding adventure daily commute 👀 w...
27055    __label__1 honor president realdonaldtrump rea...
34271    __label__1 mccormackjohn romney brings libya a...
                               ...                        
11284    __label__0 17 year incumbent cathymcmorris cow...
44732    __label__2 playavengers marvel ultimate allian...
38158    __label__1 cbo projects roughly 2 % real gdp g...
860      __label__0 seventh demdebate next week ’ still...
15795    __label__0 repcheri washington republicans cha...
Name: value, Length: 41011, dtype: object

In [599]:
train_df['value'].to_csv("tweets_train_corpus.csv",index=False)
test_df['value'].to_csv("tweets_test_corpus.csv",index=False)

In [600]:
#model = fasttext.train_supervised('tweets_train_corpus.csv',lr=0.7,epoch=50, wordNgrams=2,bucket=200000)
model = fasttext.train_supervised('tweets_train_corpus.csv')

In [601]:
model.test('tweets_test_corpus.csv')

(10253, 0.9246074319711304, 0.9246074319711304)

In [602]:
print_results(*model.test('tweets_test_corpus.csv'))

N	10253
P@1	0.925
R@1	0.925


In [537]:
#tweets_df["tweet"].to_csv("tweets_corpus.csv",index=False)

In [538]:
# Skipgram model :
#model = fasttext.train_unsupervised('tweets_corpus.csv', model='skipgram')

# or, cbow model :
#model = fasttext.train_unsupervised('tweets_corpus.csv', model='cbow')

In [539]:
#model.get_word_vector("the")

array([ 0.4146808 , -0.3514698 ,  0.99756813, -0.34905446, -0.5383298 ,
        0.26337886, -1.3346989 ,  0.2696739 ,  0.01227002, -0.11182483,
       -0.56319416,  0.5641128 ,  0.44509768, -0.6356219 ,  0.38990122,
       -0.09339877,  0.1606257 ,  0.72620386,  1.0777328 ,  0.49337825,
       -0.27959353, -0.1525427 ,  0.97561735,  0.55205953, -0.02131497,
       -0.21529667,  0.13206959,  0.16016956,  0.06136292,  0.13445513,
       -0.4657135 ,  0.6944296 ,  0.12012708, -0.78917384, -0.39463037,
        0.11260908,  0.7887055 ,  0.28799686,  0.08513959, -0.09384476,
       -0.08972653,  0.71311516,  0.16640273, -0.61693054, -0.4716776 ,
        0.3921702 , -0.48775336,  0.4053595 ,  0.18597561, -0.14477292,
        0.00245656, -0.2616625 ,  0.3431946 , -0.05408857,  0.08822475,
       -0.0115694 ,  0.2421625 ,  0.04905983,  0.44104955, -0.55255514,
        0.36913985,  0.61588585,  0.70548475,  0.4143827 ,  0.879292  ,
       -0.5183598 ,  0.5834403 , -0.11093776,  1.1930081 , -0.15

In [540]:
#model = fasttext.train_unsupervised('tweets_corpus.csv', minn=2, maxn=5, dim=300)

In [541]:
#model.get_nearest_neighbors('asparagus')

[(0.7268996238708496, 'nicaragua'),
 (0.7136082649230957, 'aoc'),
 (0.7117641568183899, 'harass'),
 (0.704505443572998, 'disarray'),
 (0.7032791972160339, 'horrendous'),
 (0.7001626491546631, 'eidmubarak'),
 (0.6917797327041626, 'agendas'),
 (0.690413773059845, 'ツ'),
 (0.6860843300819397, '⇒'),
 (0.6822501420974731, '…”')]